In [10]:
import  pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [11]:
data = pd.read_csv('Churn_Modelling.csv')

In [12]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
#preprocess the data
### Drop irrelevant columns
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [14]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [15]:
onehot_encoder_geo = onehot_encoder = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [16]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [19]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
     pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

## ANN Regression Problem

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [22]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),          # Explicit input layer
    Dense(64, activation='relu'),              # HL1
    Dense(32, activation='relu'),              # HL2
    Dense(1)             # Output layer
])

## Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from datetime import datetime
log_dir = "regressionlogs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
## set up early stopping
early_stopping_callabck = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [25]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100, 
                    callbacks=[tensorflow_callback,early_stopping_callabck])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100072.7969 - mean_absolute_error: 100072.7969 - val_loss: 98505.7109 - val_mean_absolute_error: 98505.7109
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99391.5859 - mean_absolute_error: 99391.5859 - val_loss: 96940.7266 - val_mean_absolute_error: 96940.7266
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97568.3125 - mean_absolute_error: 97568.3125 - val_loss: 92835.9375 - val_mean_absolute_error: 92835.9375
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 93050.1641 - mean_absolute_error: 93050.1641 - val_loss: 85947.1328 - val_mean_absolute_error: 85947.1328
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 85600.3984 - mean_absolute_error: 85600.3984 - val_loss: 77198.5703 - val_mean_absolute_error: 77198.5703
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 76324.4844 - mean_absolute_error: 76324.4844 - val_loss: 68233.9219 - val_mean_absolute_error: 68233.9

In [29]:
## Evaluate the model
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [30]:
log_dir

'regressionlogs/fit20250627-152716'

In [31]:
%tensorboard --logdir regressionlogs/fit20250627-152716

In [32]:
## Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51064.4883 - mean_absolute_error: 51064.4883  


In [33]:
print(f'Test MAE: {test_mae:.4f}')

Test MAE: 50331.3281


In [34]:
model.save('regression_model.h5')